adsad

In [34]:
import extract as ex

aaa = ex.get_aircrafts_per_manufacturer()

aaa

In [35]:
def get_aircrafts_per_manufacturer() -> dict[str, list[str]]:
    # TODO: Implement a function to generate a dictionary with one entry per manufacturer and a list of aircraft identifiers as values
    #print("dsaojasdajsp")
    with open("aircraft-manufaturerinfo-lookup.csv") as file:
        (next(file)) # Ignore header (aircraft_reg_code,manufacturer_serial_number,aircraft_model,aircraft_manufacturer)
        aircrafts_per_manufacturer: dict[str, list[str]] = {}
        for line in file:
            #print(line)
            linesplit = line.split(",")
            aircraft_reg_code = linesplit[0]
            manufacturer = linesplit[3][:-1]
            if manufacturer not in aircrafts_per_manufacturer.keys():
                  aircrafts_per_manufacturer[manufacturer] = []
            aircrafts_per_manufacturer[manufacturer].append(aircraft_reg_code)
    return(aircrafts_per_manufacturer)


In [36]:
result = get_aircrafts_per_manufacturer()

result

{'Airbus': ['XY-ILS',
  'XY-HZF',
  'XY-MSC',
  'XY-UNI',
  'XY-PLS',
  'XY-SBV',
  'XY-IGB',
  'XY-OOV',
  'XY-QTV',
  'XY-QKQ',
  'XY-MET',
  'XY-QJQ',
  'XY-RMP',
  'XY-FCZ',
  'XY-WZA',
  'XY-TKZ',
  'XY-JKC',
  'XY-NNT',
  'XY-IEM',
  'XY-NRN',
  'XY-OJT',
  'XY-NQQ',
  'XY-SYI',
  'XY-KEI',
  'XY-HKU',
  'XY-GUR',
  'XY-DDI',
  'XY-WUR',
  'XY-HEO',
  'XY-JRY',
  'XY-TJC',
  'XY-JPB',
  'XY-DNI',
  'XY-UHG',
  'XY-UYP',
  'XY-ECK',
  'XY-RRV',
  'XY-HWJ',
  'XY-UEG',
  'XY-RHQ',
  'XY-MEB',
  'XY-PFR',
  'XY-YKG',
  'XY-VDZ',
  'XY-GLY',
  'XY-GQC',
  'XY-ZZC',
  'XY-EFV',
  'XY-TUS',
  'XY-YTC',
  'XY-RJG',
  'XY-JZG',
  'XY-LWJ',
  'XY-EKQ',
  'XY-FYW',
  'XY-JEO',
  'XY-VTX',
  'XY-OAB',
  'XY-YIJ',
  'XY-GXJ',
  'XY-YCZ',
  'XY-MSN',
  'XY-JPQ',
  'XY-VJP',
  'XY-ETF',
  'XY-SDX',
  'XY-VGS',
  'XY-CQR',
  'XY-QTU',
  'XY-WWG',
  'XY-BBO',
  'XY-ZVO',
  'XY-ZGG',
  'XY-UQL',
  'XY-IKY',
  'XY-CLO',
  'XY-DIR',
  'XY-LRS',
  'XY-OPT',
  'XY-WBQ',
  'XY-MIL',
  'XY-WZL',
  'XY-

In [37]:


def query_utilization_baseline():
    aircrafts = get_aircrafts_per_manufacturer()
    cur = conn.cursor()
    cur.execute(f"""
        WITH atomic_data AS (
            SELECT f.aircraftregistration,
                CASE 
                    WHEN f.aircraftregistration in ('{"','".join(aircrafts.get("Airbus", []))}') THEN 'Airbus'
                    WHEN f.aircraftregistration in ('{"','".join(aircrafts.get("Boeing", []))}') THEN 'Boeing'
                    ELSE f.aircraftregistration
                    END AS manufacturer, 
                DATE_PART('year', f.scheduleddeparture)::text AS year,
                CASE WHEN f.cancelled 
                    THEN 0
                    ELSE EXTRACT(EPOCH FROM f.actualarrival-f.actualdeparture) / 3600
                    END AS flightHours,
                CASE WHEN f.cancelled 
                    THEN 0
                    ELSE 1
                    END AS flightCycles,
                CASE WHEN f.cancelled
                    THEN 1
                    ELSE 0
                    END AS cancellations,
                CASE WHEN f.cancelled
                    THEN 0
                    ELSE CASE WHEN EXTRACT(EPOCH FROM f.actualarrival - f.scheduledarrival) / 60 > 15
                        THEN 1
                        ELSE 0
                        END
                    END AS delays,
                CASE WHEN f.cancelled
                    THEN 0
                    ELSE CASE WHEN EXTRACT(EPOCH FROM f.actualarrival - f.scheduledarrival) / 60 > 15
                        THEN EXTRACT(EPOCH FROM f.actualarrival - f.scheduledarrival) / 60
                        ELSE 0
                        END
                    END AS delayedMinutes,
                0 AS scheduledOutOfService,
                0 AS unScheduledOutOfService
            FROM "AIMS".flights f
            UNION ALL
            SELECT m.aircraftregistration,           
                CASE 
                    WHEN m.aircraftregistration in ('{"','".join(aircrafts.get("Airbus", []))}') THEN 'Airbus'
                    WHEN m.aircraftregistration in ('{"','".join(aircrafts.get("Boeing", []))}') THEN 'Boeing'
                    ELSE m.aircraftregistration
                    END AS manufacturer, 
                DATE_PART('year', m.scheduleddeparture)::text AS year,
                0 AS flightHours,
                0 AS flightCycles,
                0 AS cancellations,
                0 AS delays,
                0 AS delayedMinutes,
                CASE WHEN m.programmed
                    THEN EXTRACT(EPOCH FROM m.scheduledarrival-m.scheduleddeparture)/(24*3600)
                    ELSE 0
                    END AS scheduledOutOfService,
                CASE WHEN m.programmed
                    THEN 0
                    ELSE EXTRACT(EPOCH FROM m.scheduledarrival-m.scheduleddeparture)/(24*3600)
                    END AS unScheduledOutOfService
            FROM "AIMS".maintenance m
            )
        SELECT a.manufacturer, a.year, 
            ROUND(SUM(a.flightHours)/COUNT(DISTINCT a.aircraftregistration), 2) AS FH,
            ROUND(SUM(a.flightCycles)/COUNT(DISTINCT a.aircraftregistration), 2) AS TakeOff,
            ROUND(SUM(a.scheduledOutOfService)/COUNT(DISTINCT a.aircraftregistration), 2) AS ADOSS,
            ROUND(SUM(a.unscheduledOutOfService)/COUNT(DISTINCT a.aircraftregistration), 2) AS ADOSU,
            ROUND((SUM(a.scheduledOutOfService)+SUM(a.unscheduledOutOfService))/COUNT(DISTINCT a.aircraftregistration), 2) AS ADOS,
            365-ROUND((SUM(a.scheduledOutOfService)+SUM(a.unscheduledOutOfService))/COUNT(DISTINCT a.aircraftregistration), 2) AS ADIS, -- This assumes a period of one year (as in the group by)
            ROUND(ROUND(SUM(a.flightHours)/COUNT(DISTINCT a.aircraftregistration), 2)/((365-ROUND((SUM(a.scheduledOutOfService)+SUM(a.unscheduledOutOfService))/COUNT(DISTINCT a.aircraftregistration), 2))*24), 2) AS DU,
            ROUND(ROUND(SUM(a.flightCycles)/COUNT(DISTINCT a.aircraftregistration), 2)/(365-ROUND((SUM(a.scheduledOutOfService)+SUM(a.unscheduledOutOfService))/COUNT(DISTINCT a.aircraftregistration), 2)), 2) AS DC,
            100*ROUND(SUM(delays)/ROUND(SUM(a.flightCycles), 2), 4) AS DYR,
            100*ROUND(SUM(a.cancellations)/ROUND(SUM(a.flightCycles), 2), 4) AS CNR,
            100-ROUND(100*(SUM(delays)+SUM(cancellations))/SUM(a.flightCycles), 2) AS TDR,
            100*ROUND(SUM(delayedMinutes)/SUM(delays),2) AS ADD
        FROM atomic_data a
        GROUP BY a.manufacturer, a.year
        ORDER BY a.manufacturer, a.year;
        """)
    result = cur.fetchall()
    cur.close()
    return result



In [38]:
query_utilization_baseline()

NameError: name 'conn' is not defined